In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Callbacks

In [2]:
from time import sleep
def slow_calculator(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(0.5)
        if cb: cb(i)
    return res

In [3]:
def show_progress(text, val):
    print(text, val)

In [4]:
show_value = lambda x: show_progress('Value is:', x)

In [5]:
slow_calculator(show_value)

Value is: 0
Value is: 1
Value is: 2
Value is: 3
Value is: 4


30

In [6]:
def make_show_progress(exclamation):
    _inner = lambda x: show_progress(exclamation, x)
    return _inner

In [7]:
slow_calculator(make_show_progress('Heya: '))

Heya:  0
Heya:  1
Heya:  2
Heya:  3
Heya:  4


30

In [8]:
my_exclam = make_show_progress('Cislo:')

In [9]:
slow_calculator(my_exclam)

Cislo: 0
Cislo: 1
Cislo: 2
Cislo: 3
Cislo: 4


30

In [10]:
make_show_progress

<function __main__.make_show_progress(exclamation)>

In [11]:
my_exclam

<function __main__.make_show_progress.<locals>.<lambda>(x)>

### callbacks as classes

In [12]:
def slow_calculator(cb=None):
    res = 0
    for i in range(5):
        if cb:
            cb.before_calc(i)
        res += i*i
        sleep(0.5)
        if cb and hasattr(cb, 'after_calc'):
            if cb.after_calc(i, res=res):
                break
    return res

In [13]:
class PrintCallback():
    def before_calc(self, i, *args, **kwargs):
        print('Doing calculations for round:', i)
    
    def after_calc(self, i, *args, res=None, **kwargs):
        print('Result is:', res)
        if res>10:
            print('res > 10 - breaking')
            return True
    

In [14]:
slow_calculator(PrintCallback())

Doing calculations for round: 0
Result is: 0
Doing calculations for round: 1
Result is: 1
Doing calculations for round: 2
Result is: 5
Doing calculations for round: 3
Result is: 14
res > 10 - breaking


14

In [15]:
hasattr(PrintCallback(), 'after_calc')

True

### Modifying behaviour

In [16]:
class SlowCalculator():
    def __init__(self):
        self.res = 0
    
    def before_calc(self, i, *args, **kwargs):
        print('Doing calculations for round:', i)
    
    def after_calc(self, i, *args, res=None, **kwargs):
        print('Result is:', res)
        if res>10:
            print('res > 10 - breaking')
            return True
    
    def __call__(self):
        res = 0
        for i in range(5):
            self.before_calc(i)
            res += i*i
            sleep(0.5)
            if self.after_calc(i, res=res):
                break
        return res        

In [17]:
sc = SlowCalculator()

In [18]:
sc()

Doing calculations for round: 0
Result is: 0
Doing calculations for round: 1
Result is: 1
Doing calculations for round: 2
Result is: 5
Doing calculations for round: 3
Result is: 14
res > 10 - breaking


14

In [121]:
class SlowCalculator():
    def __init__(self, cbs=None):
        self.cbs = cbs
        self.res = 0
        
    def callbacks(self, cb_name, *args, **kwargs):
        if not self.cbs: return
        for cb in self.cbs:
            cb_method = getattr(cb, cb_name, None)
            if cb_method:
                cb_method(self, *args, **kwargs)
    
    def __call__(self):
        for i in range(10):
            self.callbacks('before_calc', i)
            self.res += 1
            self.res *= i
            sleep(0.5)
            if self.callbacks('after_calc', i, res=self.res):
                break
        return self.res        

In [117]:
class ModifyingCallback():
    def before_calc(self, calc, i, *args, **kwargs):
        print('Modifying callback:', i)
    
    def after_calc(self, calc, i, *args, **kwargs):
        print('Result is:', calc.res)
        if calc.res>5 and calc.res<100:
            calc.res -= 1
            print('New results is:', calc.res)
        if calc.res>100:
            print('res > 100 - breaking')
            return True

In [118]:
callbacks = [PrintingCallback(), ModifyingCallback()]

In [119]:
sc = SlowCalculator(callbacks)

In [120]:
sc()

Printing callback: 0
Modifying callback: 0
Prinitng Result is: 0
Result is: 0
Printing callback: 1
Modifying callback: 1
Prinitng Result is: 1
Result is: 1
Printing callback: 2
Modifying callback: 2
Prinitng Result is: 4
Result is: 4
Printing callback: 3
Modifying callback: 3
Prinitng Result is: 15
Result is: 15
New results is: 14
Printing callback: 4
Modifying callback: 4
Prinitng Result is: 60
Result is: 60
New results is: 59
Printing callback: 5
Modifying callback: 5
Prinitng Result is: 300
Result is: 300
res > 100 - breaking
Printing callback: 6
Modifying callback: 6
Prinitng Result is: 1806
Result is: 1806
res > 100 - breaking
Printing callback: 7
Modifying callback: 7
Prinitng Result is: 12649
Result is: 12649
res > 100 - breaking
Printing callback: 8
Modifying callback: 8
Prinitng Result is: 101200
Result is: 101200
res > 100 - breaking
Printing callback: 9
Modifying callback: 9
Prinitng Result is: 910809
Result is: 910809
res > 100 - breaking


910809

In [77]:
class PrintingCallback():
    def before_calc(self, calc, i, *args, **kwargs):
        print('Printing callback:', i)
    
    def after_calc(self, calc, i, *args, **kwargs):
        print('Prinitng Result is:', calc.res)